# Import of NATS-Bench and ImageNet16

In [ ]:
!pip install nats_bench
!pip install xautodl

In [ ]:
import tarfile
#from xautodl.datasets.DownsampledImageNet import ImageNet16
from nats_bench import create
!wget 'https://www.dropbox.com/s/pasubh1oghex3g9/?dl=1' -O 'NATS-tss-v1_0-3ffb9-simple.tar'
#!wget 'https://www.dropbox.com/s/o2fg17ipz57nru1/?dl=1' -O ImageNet16.tar.gz
!tar xvf "NATS-tss-v1_0-3ffb9-simple.tar"

In [ ]:
!wget 'https://www.dropbox.com/s/o2fg17ipz57nru1/?dl=1' -O ImageNet16.tar.gz
file = tarfile.open('ImageNet16.tar.gz')
file.extractall('.')
file.close()

# Imports of packages

In [ ]:
# imports 
import numpy as np, collections
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from xautodl.models import get_cell_based_tiny_net, get_search_spaces, CellStructure, get_search_spaces
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.datasets.DownsampledImageNet import ImageNet16
import random
import os
import copy
import math
from scipy import stats
import time
import collections
import os, sys, time, glob, random, argparse
from copy import deepcopy

# Create the API instance for the topology search space in NATS
from nats_bench import create
api = create("/content/NATS-tss-v1_0-3ffb9-simple", 'tss', fast_mode=True, verbose=False)
#assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
batch_size = 128


# Settings for trainloader and data augmentation

In [ ]:
## https://github.com/D-X-Y/AutoDL-Projects/blob/f46486e21b71ae6459a700be720d7648b5429569/xautodl/datasets/get_dataset_with_transform.py#L99
def get_datasets(name):
    if name == "cifar10":
        mean = [x / 255 for x in [125.3, 123.0, 113.9]]
        std = [x / 255 for x in [63.0, 62.1, 66.7]]
    elif name == "cifar100":
        mean = [x / 255 for x in [129.3, 124.1, 112.4]]
        std = [x / 255 for x in [68.2, 65.4, 70.4]]
    elif name.startswith("ImageNet16"):
        mean = [x / 255 for x in [122.68, 116.66, 104.01]]
        std = [x / 255 for x in [63.22, 61.26, 65.09]]
    else:
        raise TypeError("Unknown dataset : {:}".format(name))

    # Data Argumentation
    if name == "cifar10" or name == "cifar100":
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ])
    elif name.startswith("ImageNet16"):
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
        ])
    
    if name == "cifar10": 
      trainset = dset.CIFAR10("/content/Cifar10", train=True, transform = transform, download=True)
    elif name == "cifar100": 
      trainset = dset.CIFAR100("/content/Cifar100", train=True ,transform = transform, download=True)
    elif name.startswith("ImageNet16"): 
      trainset = ImageNet16("ImageNet16", train=True, transform = transform)
    else:
      raise TypeError("Unknown dataset : {:}".format(name))
  
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True, num_workers=0, pin_memory = True)
    return trainloader

In [ ]:
datasets = ["cifar10", "cifar100", "ImageNet16-120"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Metric', 'Accuracy', 'Time'])


# SYNFLOW

In [ ]:
rm -rf zerocostnas/

In [ ]:
!git clone https://github.com/SamsungLabs/zero-cost-nas.git
import os
os.rename("zero-cost-nas", "zerocostnas")
from zerocostnas.foresight.models import nasbench2, nasbench2_ops
from zerocostnas.foresight.pruners import p_utils
from zerocostnas.foresight.pruners.measures import *
from zerocostnas.foresight.pruners import predictive
###SYNFLOW from github
#rename folder zero-cost-nas
import torch.nn as nn
import torch.nn.functional as F

def synflow(arch_str, dataset, train_loader, device, dataload='random', dataload_info=1):
  start = time.time()
  def get_nclasses(dataset):
    return 100 if dataset == 'cifar100' else 10 if dataset == 'cifar10' else 120

  net = nasbench2.get_model_from_arch_str(arch_str.tostr(), get_nclasses(dataset))
  measures = predictive.find_measures(net, train_loader, (dataload, dataload_info, get_nclasses(dataset)), device, measure_names=['synflow'])
  return math.log1p(measures['synflow']), time.time() - start ###



# REA with NASWOT without modifications

In [ ]:
def counting_forward_hook(module, inp, out):
  if isinstance(inp, tuple):
      inp = inp[0]
  inp = inp.view(inp.size(0), -1)
  x = (inp > 0).float()
  
  K = x @ x.t()
  K2 = (1.-x) @ (1.-x.t())
  global Ktemp
  Ktemp = Ktemp + K.cpu().numpy() + K2.cpu().numpy()

def init(m):
    if isinstance(m, (torch.nn.Conv2d, torch.nn.Linear)):
        torch.nn.init.xavier_normal_(m.weight)

# this is the logarithm of the determinant of K 
def hooklogdet(Ktemp, labels=None):
  s, ld = np.linalg.slogdet(Ktemp)
  return ld
  
def kernel(arch, dataset, batch_size, api, train_loader):
    start = time.time()
    index = api.query_index_by_arch(arch.tostr())
    config = api.get_net_config(index, dataset)  
    
    network = get_cell_based_tiny_net(config)
    del config
    network.apply(init)

    global Ktemp
    Ktemp = np.zeros((batch_size,batch_size))
    for name, module in network.named_modules():
      if 'ReLU' in str(type(module)):
        module.register_forward_hook(counting_forward_hook)
    
    network = network.to(device)
    data_iterator = iter(train_loader)
    x, target = next(data_iterator)
    x, target = x.to(device), target.to(device)
    network(x)
    logdet= hooklogdet(Ktemp, target)
    #print("logdet: ", logdet)
    return logdet, time.time() - start

from collections import defaultdict
def random_topology_func(op_names, max_nodes=4):
  # Return a random architecture
  def random_architecture():
      genotypes = []
      for i in range(1, max_nodes):
        xlist = []
        for j in range(i):
          node_str = "{:}<-{:}".format(i, j)
          op_name = random.choice(op_names)
          xlist.append((op_name, j))
        genotypes.append(tuple(xlist))
      
      return CellStructure(genotypes)
      
  return random_architecture

def mutate_topology_func(op_names):
    """Computes the architecture for a child of the given parent architecture.
    The parent architecture is cloned and mutated to produce the child architecture. 
    The child architecture is mutated by randomly switch one operation to another.
    """
    def mutate_architecture(parent_arch):
        child_arch = deepcopy(parent_arch)
        node_id = random.randint(0, len(child_arch.nodes) - 1)
        node_info = list(child_arch.nodes[node_id])
        snode_id = random.randint(0, len(node_info) - 1)
        xop = random.choice(op_names)
        while xop == node_info[snode_id][0]:
            xop = random.choice(op_names)
        node_info[snode_id] = (xop, node_info[snode_id][1])
        child_arch.nodes[node_id] = tuple(node_info)
        return child_arch

    return mutate_architecture

model = { "arch": None, "synflow": None}
child = { "arch": None, "synflow": None}

def regularized_evolution_kernel(
    cycles,
    population_size,
    sample_size,
    random_arch,
    mutate_arch,
    api,
    dataset,
    train_loader
):
    """Algorithm for regularized evolution (i.e. aging evolution).
    Follows "Algorithm 1" in Real et al. "Regularized Evolution for Image
    Classifier Architecture Search".
    Args:
      cycles: the number of cycles the algorithm should run for.
      population_size: the number of individuals to keep in the population.
      sample_size: the number of individuals that should participate in each tournament.
      time_budget: the upper bound of searching cost
    Returns:
      history: a list of `Model` instances, representing all the models computed
          during the evolution experiment.
    """
    population = []
    history, total_time_cost = (
        [],
        [],
    )  # Not used by the algorithm, only used to report results.
    current_best_index = []
    # Initialize the population with random models.
    while len(population) < population_size:
        model["arch"] = random_arch()
        model["metric"], total_cost = kernel( ###### 
            model["arch"], dataset, batch_size, api, train_loader #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population.append((model["metric"], model["arch"]))
        history.append((model["metric"], model["arch"]))
        #print(history)
        # print(max(history, key=lambda x: x[0]))
        total_time_cost.append(total_cost)
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1])
        )
    #print(total_time_cost)
    #print("pop: \n", population)
    
    # Carry out evolution in cycles. Each cycle produces a model and removes another.
    while len(history) < cycles:
        # Sample randomly chosen models from the current population.
        start_time, sample = time.time(), []
        while len(sample) < sample_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
            candidate = random.choice(list(population))
            sample.append(candidate)
        # The parent is the best model in the sample.
        #print("sample:\n", sample)
        parent = max(sample, key=lambda i: i[0])
        #print("p:\n", parent[1], parent[0])
        # Create the child model and store it.
        child["arch"] = mutate_arch(parent[1])
        #print("c:\n", child['arch'])
        child["metric"], total_cost = kernel( ###### 
            child["arch"], dataset, batch_size, api, train_loader #if use_proxy else api.full_train_epochs
        )
        #print("c:\n", child['arch'], child['metric'])
        # Append the info
        population[:-1] = population[1:] ##popleft
        population.append((child["metric"], child["arch"]))
        #print('pop after:\n', population)
        history.append((child["metric"], child["arch"]))
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1])
        )
        total_time_cost.append(total_cost)
        del sample, candidate, parent

    return history, current_best_index, total_time_cost

search_space = get_search_spaces("tss", "nats-bench") 
random_arch = random_topology_func(op_names = search_space) 
mutate_arch = mutate_topology_func(search_space)

# REA with Synflow without modifications


In [ ]:
model = { "arch": None, "synflow": None}
child = { "arch": None, "synflow": None}

def regularized_evolution_synflow(
    cycles,
    population_size,
    sample_size,
    random_arch,
    mutate_arch,
    api,
    dataset,
    train_loader,
    device
):

    population = []
    hist, total_time_cost = (
        [],
        [],
    )  # Not used by the algorithm, only used to report results.
    current_best_index = []
    # Initialize the population with random models.
    #print('inizio:\n')
    while len(population) < population_size:
        model["arch"] = random_arch()
        #print(model)
        model["synflow"], total_cost = synflow( ###### 
            model["arch"], dataset, train_loader, device, #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population.append(copy.deepcopy(model))
        hist.append((model["synflow"], model["arch"]))
        total_time_cost.append(total_cost)
        current_best_index.append(
            api.query_index_by_arch(max(hist, key=lambda x: x[0])[1]) ############
        )
    ## population has the same values repeated in it
    #print(population)
    #print(total_time_cost)
    # Carry out evolution in cycles. Each cycle produces a model and removes another.
    while len(hist) < cycles:
        # Sample randomly chosen models from the current population.
        start_time, sample = time.time(), []
        while len(sample) < sample_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
            candidate = random.choice(list(population))
            #print(candidate)
            sample.append(candidate) #sample is a list of dicts

        # The parent is the best model in the sample.
        #print(sample) ##always the same cause of population
        parent = max(sample, key=lambda i: i["synflow"]) ## adjust the sample to take the maximum
        #print(type(parent))
        # Create the child model and store it.
        child["arch"] = mutate_arch(parent["arch"])
        child["synflow"], total_cost = synflow( ###### 
            child["arch"], dataset, train_loader, device, #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population[:-1] = population[1:] ##popleft
        population[-1] = child
        hist.append((child["synflow"], child["arch"]))
        current_best_index.append(
            api.query_index_by_arch(max(hist, key=lambda x: x[0])[1]) ############
        )
        total_time_cost.append(total_cost)

    return hist, current_best_index, total_time_cost

search_space = get_search_spaces("tss", "nats-bench") 
random_arch = random_topology_func(op_names = search_space) 
mutate_arch = mutate_topology_func(search_space)


# REA with NASWOT with condition on SKIP_CONNECTION
Considering a minibatch of data $X = \{  {x_{i} \} }_{i=1}^{N}$ and a network, we can define an indicator variable $c_{i}$ that form a binary code that corresponds to a linear region. We can use the hamming distance between two data points $d_{H}(c_{i},c_{j})$ to measure how dissimilar the two points are. The hamming distance is the minimum number of changes needed to turn one binary code in the other. Or we could also say it is the number of positions where the two codes differ. The correspondance between binary codes can then be calculated by $K_{H}$ with $N_{A}$ as the number of ReLu activations in the network.

$$
K_{H} = \begin{bmatrix} 
    N_{A} - d_{H}(c_{1},c_{1}) & \dots & N_{A} - d_{H}(c_{1},c_{N}) \\
    \vdots & \ddots & \vdots \\
    N_{A} - d_{H}(c_{N},c_{1}) &  \dots      & N_{A} - d_{H}(c_{N},c_{N}) 
    \end{bmatrix}
$$

The scoring metric $s$ is therefore derived from the logarithm of the Kernel norm at initialization. 

$$
s = \log \lVert K_{H} \rVert
$$


In [ ]:
def random_naswotmod_func(op_names, max_nodes=4):
  # Return a random architecture
  def random_m_architecture():
      genotypes = []
      found = False
      while found == False:
        for i in range(1, max_nodes):
            xlist = []
            for j in range(i):
                op_name = random.choice(op_names)
                xlist.append((op_name, j))
            genotypes.append(tuple(xlist))
        #print(CellStructure(genotypes).tostr())
        if (CellStructure(genotypes).tostr().count('skip')>=0 & CellStructure(genotypes).tostr().count('skip')<=1): 
          #print('skip')
          found=True
        else:
          genotypes = []
      
      return CellStructure(genotypes)
      
  return random_m_architecture

def mutate_naswotmod_func(op_names):
    """Computes the architecture for a child of the given parent architecture.
    The parent architecture is cloned and mutated to produce the child architecture. 
    The child architecture is mutated by randomly switch one operation to another.
    """
    def mutate_m_architecture(parent_arch):
      found= False
      while found==False:
          child_arch = deepcopy(parent_arch)
          node_id = random.randint(0, len(child_arch.nodes) - 1)
          node_info = list(child_arch.nodes[node_id])
          snode_id = random.randint(0, len(node_info) - 1)
          xop = random.choice(op_names)
          while xop == node_info[snode_id][0]:
              xop = random.choice(op_names)
          node_info[snode_id] = (xop, node_info[snode_id][1])
          child_arch.nodes[node_id] = tuple(node_info)
          if child_arch.tostr().count('skip')==1: 
            found=True
            return child_arch
        
    return mutate_m_architecture


model = { "arch": None, "metric": None}
child = { "arch": None, "metric": None}


## P/S of: 100/2, 100/50, 20/20, 100/25, 54/16 ; best is 100/25
def regularized_evolution_kernel_m(
    cycles,
    population_size,
    sample_size,
    random_arch,
    mutate_arch,
    api,
    dataset,
    train_loader
):

    population = []
    history, total_time_cost = (
        [],
        [],
    )  # Not used by the algorithm, only used to report results.
    current_best_index = []
    # Initialize the population with random models.
    while len(population) < population_size:
        model["arch"] = random_naswotmod()
        model["metric"], total_cost = kernel( ###### 
            model["arch"], dataset, batch_size, api, train_loader #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population.append((model["metric"], model["arch"]))
        history.append((model["metric"], model["arch"]))

        total_time_cost.append(total_cost)
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1])
        )

    
    # Carry out evolution in cycles. Each cycle produces a model and removes another.
    while len(history) < cycles:
        # Sample randomly chosen models from the current population.
        start_time, sample = time.time(), []
        while len(sample) < sample_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
            candidate = random.choice(list(population))
            sample.append(candidate)
        # The parent is the best model in the sample.
        #print("sample:\n", sample)
        parent = max(sample, key=lambda i: i[0])
        #print("p:\n", parent[1], parent[0])
        # Create the child model and store it.
        child["arch"] = mutate_naswotmod(parent[1])
        #print("c:\n", child['arch'])
        child["metric"], total_cost = kernel( ###### 
            child["arch"], dataset, batch_size, api, train_loader #if use_proxy else api.full_train_epochs
        )
        #print("c:\n", child['arch'], child['metric'])
        # Append the info
        population[:-1] = population[1:] ##popleft
        population.append((child["metric"], child["arch"]))
        #print('pop after:\n', population)
        history.append((child["metric"], child["arch"]))
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1])
        )
        total_time_cost.append(total_cost)
        del sample, candidate, parent

    return history, current_best_index, total_time_cost

search_space = get_search_spaces("tss", "nats-bench") 
random_naswotmod = random_naswotmod_func(op_names = search_space) 
mutate_naswotmod = mutate_naswotmod_func(search_space)

# REA with Synflow with condition on SKIP_CONNECTION

In [ ]:
def random_synflowmod_func(op_names, max_nodes=4):
  # Return a random architecture
  def random_m_architecture():
      genotypes = []
      found = False
      while found == False:
        for i in range(1, max_nodes):
            xlist = []
            for j in range(i):
                op_name = random.choice(op_names)
                xlist.append((op_name, j))
            genotypes.append(tuple(xlist))

        if ((CellStructure(genotypes).tostr().count('skip')==1) | (CellStructure(genotypes).tostr().count('skip')==0)): #### controllo skip in MODIFICAAAAAAAAA
          print('skip')
          found=True
        else:
          genotypes = []
      
      return CellStructure(genotypes)
      
  return random_m_architecture

def mutate_synflowmod_func(op_names):
    """Computes the architecture for a child of the given parent architecture.
    The parent architecture is cloned and mutated to produce the child architecture. 
    The child architecture is mutated by randomly switch one operation to another.
    """
    def mutate_m_architecture(parent_arch):
        child_arch = deepcopy(parent_arch)
        node_id = random.randint(0, len(child_arch.nodes) - 1)
        node_info = list(child_arch.nodes[node_id])
        snode_id = random.randint(0, len(node_info) - 1)
        xop = random.choice(op_names)
        while xop == node_info[snode_id][0]:
            xop = random.choice(op_names)
        node_info[snode_id] = (xop, node_info[snode_id][1])
        child_arch.nodes[node_id] = tuple(node_info)
        return child_arch

    return mutate_m_architecture

model = { "arch": None, "synflow": None}
child = { "arch": None, "synflow": None}


## P/S of: 100/2, 100/50, 20/20, 100/25, 54/16 ; best is 100/25
def regularized_evolution_synflow_m(
    cycles,
    population_size,
    sample_size,
    random_arch,
    mutate_arch,
    api,
    dataset,
    train_loader,
    device
):

    population = []
    history, total_time_cost = (
        [],
        [],
    )  # Not used by the algorithm, only used to report results.
    current_best_index = []
    # Initialize the population with random models.
    while len(population) < population_size:
        model["arch"] = random_synflowmod()
        model["synflow"], total_cost = synflow( ###### 
            model["arch"], dataset, train_loader, device, #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population.append(copy.deepcopy(model))
        history.append((model["synflow"], model["arch"]))
        total_time_cost.append(total_cost)
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1]) ############
        )
    ## population has the same values repeated in it
    #print(population)
    #print(total_time_cost)
    # Carry out evolution in cycles. Each cycle produces a model and removes another.
    while len(history) < cycles:
        # Sample randomly chosen models from the current population.
        start_time, sample = time.time(), []
        while len(sample) < sample_size:
            # Inefficient, but written this way for clarity. In the case of neural
            # nets, the efficiency of this line is irrelevant because training neural
            # nets is the rate-determining step.
            candidate = random.choice(list(population))
            #print(candidate)
            sample.append(candidate) #sample is a list of dicts

        # The parent is the best model in the sample.
        #print(sample) ##always the same cause of population
        parent = max(sample, key=lambda i: i["synflow"]) ## adjust the sample to take the maximum
        #print(type(parent))
        # Create the child model and store it.
        child["arch"] = mutate_synflowmod(parent["arch"])
        child["synflow"], total_cost = synflow( ###### 
            child["arch"], dataset, train_loader, device, #if use_proxy else api.full_train_epochs
        )
        # Append the info
        population[:-1] = population[1:] ##popleft
        population[-1] = child
        history.append((child["synflow"], child["arch"]))
        current_best_index.append(
            api.query_index_by_arch(max(history, key=lambda x: x[0])[1]) ############
        )
        total_time_cost.append(total_cost)
        del sample, candidate, parent

    return history, current_best_index, total_time_cost

search_space = get_search_spaces("tss", "nats-bench") 
random_synflowmod = random_synflowmod_func(op_names = search_space) 
mutate_synflowmod = mutate_synflowmod_func(search_space)

# Run with REA + NASWOT 

In [ ]:
Ktemp = []
for dataset in datasets:
  for run in range(10):
    train_loader = get_datasets(dataset)
    ## REA 
    history, current_best_index, total_times = regularized_evolution_kernel(cycles = 200,
                                                                    population_size = 50, 
                                                                    sample_size = 10, 
                                                                    random_arch= random_arch, 
                                                                    mutate_arch=mutate_arch,  
                                                                    api=api, 
                                                                    dataset= dataset,
                                                                    train_loader = train_loader,
                                                                    )
    best_arch = max(history, key=lambda x: x[0])[1]
    best_score = max(history, key=lambda x: x[0])[0]
    

    index = api.query_index_by_arch(best_arch.tostr()) 

    csv_dict = {'Dataset': dataset, 'Network_index': index, 'Metric': best_score, 'Accuracy': api.get_more_info(int(index), dataset, hp="200", is_random=False)["test-accuracy"] , 'Time': sum(total_times)}

    df_dict = pd.DataFrame([csv_dict])
    result = pd.concat([result, df_dict], ignore_index=True)

    result.to_csv(f'outKERNELwithoutMOD10runs_{dataset}.csv', mode='a', index=False, header=False )
    result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Metric', 'Accuracy', 'Time'])

# Run with REA + NASWOT with skip filter

In [ ]:
Ktemp = []
for dataset in datasets:
  for run in range(10):
    train_loader = get_datasets(dataset)
    ## REA 
    history, current_best_index, total_times = regularized_evolution_kernel_m(cycles = 200,
                                                                    population_size = 50, 
                                                                    sample_size = 10, 
                                                                    random_arch= random_naswotmod, 
                                                                    mutate_arch=mutate_naswotmod,  
                                                                    api=api, 
                                                                    dataset= dataset,
                                                                    train_loader = train_loader,
                                                                    )
    best_arch = max(history, key=lambda x: x[0])[1]
    best_score = max(history, key=lambda x: x[0])[0]
    

    index = api.query_index_by_arch(best_arch.tostr()) 

    csv_dict = {'Dataset': dataset, 'Network_index': index, 'Metric': best_score, 'Accuracy': api.get_more_info(int(index), dataset, hp="200", is_random=False)["test-accuracy"] , 'Time': sum(total_times)}

    df_dict = pd.DataFrame([csv_dict])
    result = pd.concat([result, df_dict], ignore_index=True)

    result.to_csv(f'outKERNELwithMOD10runs_{dataset}.csv', mode='a', index=False, header=False )
    result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Metric', 'Accuracy', 'Time'])

# Run with REA + Synflow 

In [ ]:
Ktemp = []
for dataset in datasets:
  for run in range(10):
    train_loader = get_datasets(dataset)
    ## REA 
    history, current_best_index, total_times = regularized_evolution_synflow(cycles = 200,
                                                                    population_size = 50, 
                                                                    sample_size = 10, 
                                                                    random_arch= random_arch, 
                                                                    mutate_arch=mutate_arch,  
                                                                    api=api, 
                                                                    dataset= dataset,
                                                                    train_loader = train_loader,
                                                                    device=device)
    best_arch = max(history, key=lambda x: x[0])[1]
    best_score = max(history, key=lambda x: x[0])[0]
    

    index = api.query_index_by_arch(best_arch.tostr()) 

    csv_dict = {'Dataset': dataset, 'Network_index': index, 'Synflow': best_score, 'Accuracy': api.get_more_info(int(index), dataset, hp="200", is_random=False)["test-accuracy"] , 'Time': sum(total_times)}

    df_dict = pd.DataFrame([csv_dict])
    result = pd.concat([result, df_dict], ignore_index=True)

    result.to_csv(f'outSYNFLOWwithoutMOD10runs_{dataset}.csv', mode='a', index=False, header=False )
    result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Synflow', 'Accuracy', 'Time'])

# Run with REA + Synflow with skip filter


In [ ]:
Ktemp = []
for dataset in datasets:
  for run in range(10):
    train_loader = get_datasets(dataset)
    ## REA 
    history, current_best_index, total_times = regularized_evolution_synflow_m(cycles = 200,
                                                                    population_size = 50, 
                                                                    sample_size = 10, 
                                                                    random_arch= random_synflowmod, 
                                                                    mutate_arch=mutate_synflowmod,  
                                                                    api=api, 
                                                                    dataset= dataset,
                                                                    train_loader = train_loader,
                                                                    device=device)
    best_arch = max(history, key=lambda x: x[0])[1]
    best_score = max(history, key=lambda x: x[0])[0]
    

    index = api.query_index_by_arch(best_arch.tostr()) 

    csv_dict = {'Dataset': dataset, 'Network_index': index, 'Synflow': best_score, 'Accuracy': api.get_more_info(int(index), dataset, hp="200", is_random=False)["test-accuracy"] , 'Time': sum(total_times)}

    df_dict = pd.DataFrame([csv_dict])
    result = pd.concat([result, df_dict], ignore_index=True)

    result.to_csv(f'outSYNFLOWwithMOD10runs_{dataset}.csv', mode='a', index=False, header=False )
    result = pd.DataFrame(columns=['Dataset', 'Network_index', 'Synflow', 'Accuracy', 'Time'])

# PLOTS for Cifar10

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

!git clone https://github.com/jjguglielmi/Neural-Architecture-Search-without-Training.git


def func(string):
  count = ''
  for i in range(0,3):
    if(i!=0): count = count + ' '
    tot = string.split('+')[i].count('skip')
    count = count + str(tot)
  return count

# if you are running on Google Colab, left the variable 'csvs' like that; instead if you are not, remove '/content/'
csvs = ['/content/Neural-Architecture-Search-without-Training/pre-trained-architectures/out_c10_200.csv', 
        '/content/Neural-Architecture-Search-without-Training/pre-trained-architectures/out_c100_200.csv', 
        '/content/Neural-Architecture-Search-without-Training/pre-trained-architectures/out_IN_200.csv']

header=['Dataset', 'sample', 'index', 'LogDet', 'TestAccuracy', 'Time']
score = '/content/Neural-Architecture-Search-without-Training/pre-trained-architectures/scores.csv'

In [ ]:
from csv import reader
import csv
headerS=['Dataset', 'sample', 'index', 'LogDet', 'TestAccuracy', 'Time']
headerSkip = ['Dataset', 'Arch', 'count_skip', 'pos_skip', 'TestAccuracy', 'index']

with open(score, 'r') as read_obj:
  csv_reader = reader(read_obj)
  with open('skips.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(headerSkip)
    for row in csv_reader:
      for ds in datasets:
        if row[0] == ds:
          arch_str= api.get_net_config(int(row[2]), row[0])['arch_str']
          l= [row[0],arch_str, int(arch_str.count('skip')), func(arch_str), row[4], row[2]]
          writer.writerow(l)

df= pd.read_csv('/content/skips.csv', names=headerSkip, skiprows=1)
sns.set_style("white")
df=df.astype({'count_skip': int, 'TestAccuracy': float, 'Arch': str, 'pos_skip':str})
x0 = df.loc[(df['count_skip']==0) & (df['Dataset']=='cifar10'),  ['TestAccuracy']]
xS= df.loc[(df['count_skip']>0) & (df['Dataset']=='cifar10'), ['TestAccuracy']]
x0['Skip/notSkip'] = 'notSkip'
xS['Skip/notSkip'] = 'Skip'
xTot= pd.concat([x0, xS])
#print(df)
plt.figure(figsize=(8,5), dpi= 80)
sns.histplot(xTot, x='TestAccuracy', hue='Skip/notSkip', element='bars')

plt.savefig(f'histSkip_noSkipScorescifar10.png')

skip_df = df
skip_count = skip_df['Arch'].str.count('skip')
skip_df['count_skip'] = skip_count
acc_count= skip_df[['TestAccuracy', 'count_skip', 'Dataset']].copy()
plt.figure(figsize=(8,5), dpi= 80)
sns.histplot(acc_count, x='TestAccuracy', hue='count_skip', element='poly')
plt.title(f'Skip distribution for {ds}')
plt.savefig(f'histSkipcifar10.png')
df_skips= acc_count.loc[(acc_count['count_skip']<=3) & (acc_count['TestAccuracy']>20) & (acc_count['Dataset']=='cifar10'), ['TestAccuracy', 'count_skip']]
plt.figure(figsize=(8,5), dpi= 80)
plt.title(f'Distribution of architectures based on the number of skips ({ds})')
sns.histplot(df_skips, x='TestAccuracy', hue='count_skip', element='poly', palette = 'copper_r')
plt.savefig(f'histHighAccCifar10.png')
df_skips.groupby("count_skip").agg(["mean", "std"]).to_csv(f'comparisonMeanSTDSkipscifar10.csv') ## table mean/std of arch with 1/2/3 skips without outliers (acc>20)
tot= skip_df.loc[(skip_df['count_skip']<=3) & (acc_count['TestAccuracy']>20) & (acc_count['Dataset']=='cifar10'), ['Arch', 'TestAccuracy', 'count_skip']]
totPosSkip = tot.copy()
totPosSkip['pos_countSkip'] = totPosSkip['Arch'].apply(lambda x: str(func(x)))
new = totPosSkip.filter(['TestAccuracy', 'count_skip', 'pos_countSkip'])
sorted = new.sort_values('TestAccuracy', ascending=False)
##sorted.head(5).to_latex()
sorted.head(10).to_csv(f'first10SortedArchwithSkipPoscifar10.csv')